In [1]:
import re

In [19]:
headerExample = '''
########################################################
#!version=2.0
#Bond Fluctuation Model - simulation data file
#Wed May 27 05:56:55 2020
#monomer numbering starts at 1

########################################################

# meta-data:
# 
# registered features: 
# 
# FeatureMoleculesIO
#       Number of monomers: 133100
#       max connectivity: 4
# FeatureBox
# FeatureBondset<FastBondset>
# FeatureAttributes<int>
# FeatureLatticePowerOfTwo<unsigned char>
# FeatureExcludedVolumeSc<FeatureLatticePowerOfTwo<unsigned char> >
# FeatureNNInteractionSc<FeatureLatticePowerOfTwo>
# FeatureBoltzmann
# 
# not registered features: 
# 
# #!version
# endoffile
# 
########################################################

!number_of_monomers=133100

!bonds
17 30
9 42
49 62


!box_x=128

!box_y=128

!box_z=128

!periodic_x=1

!periodic_y=1

!periodic_z=1

!set_of_bondvectors
2 0 0:17
0 0 2:18
0 2 0:19
-2 0 0:20


## nearest neighbor interactions between types in kT (default 0.0kT)
!nn_interaction 3 4 0.8
!nn_interaction 3 5 0.8



!mcs=1000000000
145 -36 -27 X^X^].a#.4w^Tx7^T&B-TQ?c+)9J^_
144 -43 -32 ^Y%^^=*U'mI+D
147 -38 -35 \^X,^V?E&^Qk#^Z]$)^\Y"

'''

# set up regular expressions
# use https://regexper.com to visualise these if required
rx_dict = {
    'mcs': re.compile(r'!mcs=(?P<mcs>\d+)\n'),
    'number_of_monomers': re.compile(r'!number_of_monomers=(?P<number_of_monomers>\d+)\n'),
    'box_x': re.compile(r'!box_x=(?P<box_x>\d+)\n'),
    'box_y': re.compile(r'!box_y=(?P<box_y>\d+)\n'),
    'box_z': re.compile(r'!box_z=(?P<box_z>\d+)\n'),
    'periodic_x': re.compile(r'!periodic_x=(?P<periodic_x>\d+)\n'),
    'periodic_y': re.compile(r'!periodic_y=(?P<periodic_y>\d+)\n'),
    'periodic_z': re.compile(r'!periodic_z=(?P<periodic_z>\d+)\n'),
    'nn_interaction': re.compile(r'!nn_interaction (\d) (\d) (?P<nn_interaction>\d+\.\d+)\n'),
    'feature_name': re.compile(r'# Feature(?P<feature_name>.*)\n'),
}

In [20]:
for key, rx in rx_dict.items():
        match = rx.search(headerExample)
        if match:
            print(key, match.group(key))

mcs 1000000000
number_of_monomers 133100
box_x 128
box_y 128
box_z 128
periodic_x 1
periodic_y 1
periodic_z 1
nn_interaction 0.8
feature MoleculesIO


In [ ]:
# Some code to start with from 
# https://www.vipinajayakumar.com/parsing-text-with-python/#parsing-text-in-standard-format

def _parse_line(line):
    """ Apply the regex dictionary on every line to find key value pairs
    
    Parameters:
        line (str): the actual line of the file or a string
        
    Returns:
        key and value of the regex dictionary defined in rx_dict if something was found
        None, None otherwise
    """
    for key, rx in rx_dict.items():
        match = rx.search(line)
        if match:
            return key, match.group(key)
    # if there are no matches
    return None, None


def parse_file(filepath):
    """Parse contant of a given bfm file

    Parameters:
        filepath (str): path to the bfm file
        
    Returns:
        data (pd.DataFrame): parsed data summarized in a pandas DataFrame

    """

    data = []  # create an empty list to collect the data
    # open the file and read through it line by line
    with open(filepath, 'r') as file_object:
        line = file_object.readline()
        while line:
            # at each line check for a match with a regex
            key, match = _parse_line(line)

            # find mcs to stop after header
            if key == 'mcs':
                print("reached mcs command, stop reading")
                # close file

            # extract grade
            if key == 'number_of_monomers':
                print("Number of monomers = ", match)
                
# --------------------------------------------- not modified yet ------------------------------------------------#
            # identify a table header 
            if key == 'name_score':
                # extract type of table, i.e., Name or Score
                value_type = match.group('name_score')
                line = file_object.readline()
                # read each line of the table until a blank line
                while line.strip():
                    # extract number and value
                    number, value = line.strip().split(',')
                    value = value.strip()
                    # create a dictionary containing this row of data
                    row = {
                        'School': school,
                        'Grade': grade,
                        'Student number': number,
                        value_type: value
                    }
                    # append the dictionary to the data list
                    data.append(row)
                    line = file_object.readline()

            line = file_object.readline()

        # create a pandas DataFrame from the list of dicts
        data = pd.DataFrame(data)
        # set the School, Grade, and Student number as the index
        data.set_index(['School', 'Grade', 'Student number'], inplace=True)
        # consolidate df to remove nans
        data = data.groupby(level=data.index.names).first()
        # upgrade Score from float to integer
        data = data.apply(pd.to_numeric, errors='ignore')
    return data